In [21]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


DataSchema = [
  StructField('agmbgm', StringType(), True),
  StructField('business_unit', StringType(), True),
  StructField('dateTime', TimestampType(), True),
  StructField('chargetype', StringType(), True),
  StructField('cluster_cost', BooleanType(), True),
  StructField('cost', DoubleType(), True),
  StructField('cost_location', StringType(), True),
  StructField('currency', StringType(), True),
  StructField('departmentCode', StringType(), True),
  StructField('environment', StringType(), True),
  StructField('exec_owner', StringType(), True),
  StructField('finance_owner', StringType(), True),
  StructField('globally_shared', BooleanType(), True),
  StructField('is_cie_project', BooleanType(), True),
  StructField('meter', StringType(), True),
  StructField('meter_subcategory', StringType(), True),
  StructField('pricingmodel', StringType(), True),
  StructField('product', StringType(), True),
  StructField('product_owner', StringType(), True),
  StructField('project_name', StringType(), True),
  StructField('res_name', StringType(), True),
  StructField('resource_group', StringType(), True),
  StructField('run_date', StringType(), True),
  StructField('service_name', StringType(), True),
  StructField('service_tier', StringType(), True),
  StructField('sleeve', StringType(), True),
  StructField('unique_id', StringType(), True),
  StructField('to_date', IntegerType(), True),
  StructField('to_hour', IntegerType(), True)
]
FSchema = StructType(fields=DataSchema)

In [23]:
spark = SparkSession.builder.appName("Finalscript").master("local[*]").getOrCreate()
AzureCostdf = spark.read.csv("/content/drive/MyDrive/PySpark/AzureCost220401.csv",sep=',',header=False,schema=FSchema)
SumEnvdf = AzureCostdf.filter(AzureCostdf.cluster_cost == 'True').groupBy('environment').agg({'cost':'sum'})
SumEnvdf.show(truncate= False)

+---------------------------------+------------------+
|environment                      |sum(cost)         |
+---------------------------------+------------------+
|Federal Prod USGovArizona        |807.4449759136766 |
|US Commercial Stage WestUS2      |297.30310516298334|
|US Commercial QA EastUS          |310.18544691566916|
|Federal Stage USGovArizona       |520.5189347155218 |
|US Government Stage USGovVirginia|776.367586194089  |
|US Commercial Dev EastUS         |854.2736493374249 |
|US Commercial Stage Eastus       |618.0993042693704 |
|Australia Prod AustraliaCentral  |409.2804430311417 |
|US Commercial Prod WestUS2       |174.28526361260012|
|Federal Stage USGovVirginia      |536.7531079555896 |
|US Commercial QA WestUS2         |123.0367954335855 |
|UK Prod UKSouth                  |288.2568591381056 |
|US Commercial Prod EastUS        |533.8389104732022 |
|US Government Prod USGovVirginia |1365.2241911875672|
|US Commercial Dev WestUS2        |256.54650442932905|
|Federal P

In [24]:
SumEnvdf = SumEnvdf.select(col('environment'),col('sum(cost)').alias('environment_wise_cost'))
SumEnvdf = SumEnvdf.filter(SumEnvdf.environment == 'US Commercial Dev EastUS')
SumEnvdf.show()

+--------------------+---------------------+
|         environment|environment_wise_cost|
+--------------------+---------------------+
|US Commercial Dev...|    854.2736493374249|
+--------------------+---------------------+



In [25]:
UsageSchema = StructType([
StructField('namespace', StringType(), True),
StructField('timestamp', StringType(), True),
StructField('kubernetes_pod_name_keyword', StringType(),True),
StructField('kubernetes_pod_cpu_usage_node_pct', DoubleType(), True),
StructField('kubernetes_container_cpu_limit_cores', DoubleType(),True),
StructField('kubernetes_container_cpu_request_cores', DoubleType(), True),
StructField('kubernetes_pod_cpu_usage_limit_pct', DoubleType(),True),
StructField('kubernetes_pod_memory_usage_bytes', DoubleType(), True),
StructField('kubernetes_pod_cpu_usage_nanocores', DoubleType(),True),
StructField('key', StringType(), True),
StructField('environment_name', StringType(), True),
StructField('to_date', IntegerType(), True)
])

In [26]:
Usagedf = spark.read.csv('/content/drive/MyDrive/PySpark/UsageDevEastus220401.csv', sep=",", header=False, schema=UsageSchema)

In [27]:
Usagedf = Usagedf.groupBy('environment_name','namespace','kubernetes_pod_name_keyword').mean('kubernetes_pod_cpu_usage_nanocores')
Usagedf.sort('namespace').show(truncate = False)

+------------------------+-----------------------+--------------------------------------------------+---------------------------------------+
|environment_name        |namespace              |kubernetes_pod_name_keyword                       |avg(kubernetes_pod_cpu_usage_nanocores)|
+------------------------+-----------------------+--------------------------------------------------+---------------------------------------+
|US Commercial Dev EastUS|accnext-dev            |test-msivms-core-0-899f44fb7-dfhc2                |5.0066515375E7                         |
|US Commercial Dev EastUS|accnext-dev            |test-msivms-recorder-0-579ff5849f-45q85           |4.201701253819445E7                    |
|US Commercial Dev EastUS|accnext-dev            |test-msivms-core-2-6554bbf69-g9nmz                |3.9127851965277776E7                   |
|US Commercial Dev EastUS|accnext-dev            |test-msivms-recorder-1-7dc79db9c6-hf7v9           |4.474252466319445E7                    |
|US Co

In [28]:
Nameusagedf = Usagedf.groupBy('environment_name','namespace').sum('avg(kubernetes_pod_cpu_usage_nanocores)')
Nameusagedf = Nameusagedf.select(col("environment_name"),col("namespace"),col("sum(avg(kubernetes_pod_cpu_usage_nanocores))").alias('nanocores_usage'))
Nameusagedf.sort('namespace').show(truncate = False)

+------------------------+------------------------+---------------------+
|environment_name        |namespace               |nanocores_usage      |
+------------------------+------------------------+---------------------+
|US Commercial Dev EastUS|accnext-dev             |2.2554439744097224E8 |
|US Commercial Dev EastUS|acre-access-reviews-dev |1.2613341368055556E7 |
|US Commercial Dev EastUS|alphasi500-dev          |0.0                  |
|US Commercial Dev EastUS|ams-dev                 |2.134856019965278E8  |
|US Commercial Dev EastUS|analytics-services-dev  |5.576962904381945E9  |
|US Commercial Dev EastUS|apollo-dev              |5.0912350749128294E8 |
|US Commercial Dev EastUS|appsec-dev              |1.1610583125780592E10|
|US Commercial Dev EastUS|astroupgrademanager-dev |0.0                  |
|US Commercial Dev EastUS|avo-indoor-maps-dev     |71508.61458333333    |
|US Commercial Dev EastUS|aware-streaming-dev     |4.172596148611111E7  |
|US Commercial Dev EastUS|axs-cloud-de

In [29]:
TotalUsagedf = Usagedf.groupby('environment_name').sum('avg(kubernetes_pod_cpu_usage_nanocores)')
TotalUsagedf = TotalUsagedf.select(col("environment_name"),col("sum(avg(kubernetes_pod_cpu_usage_nanocores))").alias('total_nanocores_usage'))
TotalUsagedf.show()

+--------------------+---------------------+
|    environment_name|total_nanocores_usage|
+--------------------+---------------------+
|US Commercial Dev...| 4.343245449658559E11|
|     enviroment_name|                 null|
+--------------------+---------------------+



In [30]:
nanodf = TotalUsagedf.join(Nameusagedf,['environment_name'],'inner');
nanodf.show()

+--------------------+---------------------+--------------------+--------------------+
|    environment_name|total_nanocores_usage|           namespace|     nanocores_usage|
+--------------------+---------------------+--------------------+--------------------+
|US Commercial Dev...| 4.343245449658559E11|         mercury-dev|1.5391453694444444E7|
|US Commercial Dev...| 4.343245449658559E11|           ngcmf-dev| 2.461207590163546E9|
|US Commercial Dev...| 4.343245449658559E11|          appsec-dev|1.161058312578059...|
|US Commercial Dev...| 4.343245449658559E11|acre-access-revie...|1.2613341368055556E7|
|US Commercial Dev...| 4.343245449658559E11|              rwqm68|                 0.0|
|US Commercial Dev...| 4.343245449658559E11|cie-interactive-t...|   2087944.048611111|
|US Commercial Dev...| 4.343245449658559E11|virtualpartner-bo...|1.2741297210557911E9|
|US Commercial Dev...| 4.343245449658559E11|         ccdrive-dev|1.196289083449170...|
|US Commercial Dev...| 4.343245449658559E11

In [31]:
pudf = nanodf.select(col("environment_name").alias('environment'),col("namespace"),((col("nanocores_usage")/col("total_nanocores_usage"))*100).alias("percentage_usage"))
pudf.show(truncate = False)

+------------------------+-------------------------+---------------------+
|environment             |namespace                |percentage_usage     |
+------------------------+-------------------------+---------------------+
|US Commercial Dev EastUS|mercury-dev              |0.0035437678742412386|
|US Commercial Dev EastUS|ngcmf-dev                |0.5666747639963641   |
|US Commercial Dev EastUS|appsec-dev               |2.673250512860918    |
|US Commercial Dev EastUS|acre-access-reviews-dev  |0.0029041281489276976|
|US Commercial Dev EastUS|rwqm68                   |0.0                  |
|US Commercial Dev EastUS|cie-interactive-tools-dev|4.807336064267916E-4 |
|US Commercial Dev EastUS|virtualpartner-bot-dev   |0.2933589031114868   |
|US Commercial Dev EastUS|ccdrive-dev              |2.7543667455939334   |
|US Commercial Dev EastUS|cgw-dev                  |0.010871733502454392 |
|US Commercial Dev EastUS|silk-dev                 |0.0                  |
|US Commercial Dev EastUS

In [32]:
EnvUsagedf = SumEnvdf.join(pudf,['environment'],'inner')
EnvUsagedf.show(truncate = False)

+------------------------+---------------------+-------------------------+---------------------+
|environment             |environment_wise_cost|namespace                |percentage_usage     |
+------------------------+---------------------+-------------------------+---------------------+
|US Commercial Dev EastUS|854.2736493374249    |mercury-dev              |0.0035437678742412386|
|US Commercial Dev EastUS|854.2736493374249    |ngcmf-dev                |0.5666747639963641   |
|US Commercial Dev EastUS|854.2736493374249    |appsec-dev               |2.673250512860918    |
|US Commercial Dev EastUS|854.2736493374249    |acre-access-reviews-dev  |0.0029041281489276976|
|US Commercial Dev EastUS|854.2736493374249    |rwqm68                   |0.0                  |
|US Commercial Dev EastUS|854.2736493374249    |cie-interactive-tools-dev|4.807336064267916E-4 |
|US Commercial Dev EastUS|854.2736493374249    |virtualpartner-bot-dev   |0.2933589031114868   |
|US Commercial Dev EastUS|854.

In [33]:
Finaldf = EnvUsagedf.select(col('namespace'),col('percentage_usage'),((col('environment_wise_cost')*col('percentage_usage'))/100).alias('cost'))
Finaldf.show(truncate = False)

+-------------------------+---------------------+---------------------+
|namespace                |percentage_usage     |cost                 |
+-------------------------+---------------------+---------------------+
|mercury-dev              |0.0035437678742412386|0.030273475143327917 |
|ngcmf-dev                |0.5666747639963641   |4.840953186265979    |
|appsec-dev               |2.673250512860918    |22.836874712148393   |
|acre-access-reviews-dev  |0.0029041281489276976|0.024809201519280046 |
|rwqm68                   |0.0                  |0.0                  |
|cie-interactive-tools-dev|4.807336064267916E-4 |0.004106780523213566 |
|virtualpartner-bot-dev   |0.2933589031114868   |2.506087807266739    |
|ccdrive-dev              |2.7543667455939334   |23.529829313721763   |
|cgw-dev                  |0.010871733502454392 |0.09287435453765658  |
|silk-dev                 |0.0                  |0.0                  |
|cc-routing-dev           |0.20311463174880112  |1.7351547769787